# Task

In [5]:
# import libraries
import pandas as pd

### EDA (Exploratory Data Analysis)

In [ ]:
# load data into pandas DataFrame
df_initial = pd.read_csv('./amplitude_test.csv', index_col=0)
df_initial['event_id'] = df_initial.index

df_initial.head(1)

,client_event_time,device_id,session_id,language,country,region,platform,os_name,device_type,device_family,event_type,event_properties,user_properties,event_id
0,2023-10-24 15:42:24.506,84bfeea2-f6ec-47c1-b9d0-791d1ac5c616,1698162134395,English,United States,North Carolina,Web,Chrome,NaN,K,session_end,{},{'fbclid': 'IwAR295tUiGCljtUzVYpJ6BMKwmbhOsfWO...,0
1,2023-10-25 16:19:00.428,84bfeea2-f6ec-47c1-b9d0-791d1ac5c616,1698250740429,English,United States,North Carolina,Web,Chrome,NaN,K,session_start,{},{'fbclid': 'IwAR295tUiGCljtUzVYpJ6BMKwmbhOsfWO...,1
2,2023-10-25 16:19:01.809,84bfeea2-f6ec-47c1-b9d0-791d1ac5c616,1698250740429,English,United States,North Carolina,Web,Chrome,NaN,K,[Amplitude] Page Viewed,{'[Amplitude] Page URL': 'https://example.com/...,{'fbclid': 'IwAR295tUiGCljtUzVYpJ6BMKwmbhOsfWO...,2
3,2023-10-25 16:19:01.878,84bfeea2-f6ec-47c1-b9d0-791d1ac5c616,1698250740429,English,United States,North Carolina,Web,Chrome,NaN,K,Configurator start,"{'type': 'Hoodie', 'point.header': 'https://ex...",{'fbclid': 'IwAR295tUiGCljtUzVYpJ6BMKwmbhOsfWO...,3
4,2023-10-25 16:19:01.880,84bfeea2-f6ec-47c1-b9d0-791d1ac5c616,1698250740429,English,United States,North Carolina,Web,Chrome,NaN,K,session_end,{},{'fbclid': 'IwAR295tUiGCljtUzVYpJ6BMKwmbhOsfWO...,4
5,2023-11-06 00:01:23.464,84bfeea2-f6ec-47c1-b9d0-791d1ac5c616,1699228883465,English,United States,North Carolina,Web,Chrome,NaN,K,session_start,{},{'fbclid': 'IwAR295tUiGCljtUzVYpJ6BMKwmbhOsfWO...,5


In [7]:
len(df_initial['session_id'].unique())

17599

In [30]:
df_session = df_initial.copy()
df_session = df_session[df_session['session_id'] == 1700908779174]
df_session = df_session.sort_values(by='client_event_time')
df_session.head(2)

,client_event_time,device_id,session_id,language,country,region,platform,os_name,device_type,device_family,event_type,event_properties,user_properties,event_id
621933,2023-11-25 10:39:39.173,28ad24a0-4855-4ba3-a4e8-08d3c87c4e5a,1700908779174,Serbian,Serbia,Belgrade,Web,Instagram,Samsung Galaxy S20 FE,Samsung Phone,session_start,{},{},621933
621934,2023-11-25 10:39:39.325,28ad24a0-4855-4ba3-a4e8-08d3c87c4e5a,1700908779174,Serbian,Serbia,Belgrade,Web,Instagram,Samsung Galaxy S20 FE,Samsung Phone,[Amplitude] Page Viewed,{'[Amplitude] Page URL': 'https://example.com/...,{'fbclid': 'PAAaZQoV0BVodHokL3W9m74EZH0hK_2gCY...,621934


### Построение Воронки (Funnel)  
Для того, чтобы построить воронку конверсии на разных этапах оформления заказа мы выберем для каждой сессии те этапы, на которые вышел пользователь.  
Для этого мы выберем только колонки *session_id* и *event_type* и затем уберем дубликаты. Таким образом мы получим все Этапы для каждой сессии.  
В воронке мы будем считать количество сессий для каждого этапа воронки.

In [21]:
# take columns: session_id, event_type:  
df_funnel = df_initial.copy()
df_funnel = df_funnel[['session_id', 'event_type']]
# drop duplicates:
df_funnel = df_funnel.drop_duplicates()
# save to csv file:
df_funnel.to_csv('./df_funnel_session_event.csv', sep='|', index=False)
df_funnel

,session_id,event_type
0,1698162134395,session_end
1,1698250740429,session_start
2,1698250740429,[Amplitude] Page Viewed
3,1698250740429,Configurator start
4,1698250740429,session_end
...,...,...
815414,1701847657262,Next button clicked
815417,1701847657262,basket config pressed
815418,1701847657262,product add
815419,1701847657262,order view


![](./pic/1.png)

В Воронке мы можем отметить следующее:
- В продукте достаточно высокая конверсия в начало использования "Конфигуратора" (более половины всех пользователей, которым показали страницу, запустили конфигуратор). Изходя из этого делаем вывод о том, что сам конфигуратов представляется интересным пользователям
- Конферсия в работу с конфигуратором также достаточно высокая
- Далее мы видим существенное снижение конверсии (только 9,3% пользователей долшщи до Home page магазина)

Продолжим рассмотрение воронки далее:
- В продукте хорошая конверсия из *Home page* в *basket config*
- Также далее коверсия тоже очень хорошая вплоть до *Random button*. Возникает вопрос к функционалу такой кнопки. Возможно она сбивает пользователей. Возможно она стоит слишком поздно в Client Journey или с ней возникают другие проблемы у пользователей. Возможно стоит провести тест, чтобы ее убрать.
- Далее по воронке существенное снижение возникает на этапе перехода в корзину (в целом это логично, но с этим скорее всего есть возможность поработать - сделать напоминание о корзине, подсветить ее или использовать другие методы)  
Таким образом, только 2,8% пользователей доходит до корзины.

![](./pic/4.png)

Сгруппируем некоторый Events и уберем events, которые не относятся к воронке Target action (payment)

![](./pic/5.png)

Мы видим, что Клиентский путь остается достаточно сложным и оставляет много вопросов к необходимости тех или иных этапов в воронке: Random button, product card submitted etc.

### Funnel в различных разрезах

Посмотрим на воронку продукта в различных разрезах:
- Язык (language)
- Страны (country)
- Регион (region)
- OS
- Утройства (device_type)
- device_family


In [29]:
# device type:
df_funnel_device = df_initial.copy()
df_funnel_device = df_funnel_device[['session_id', 'event_type', 'device_type']]
df_funnel_device = df_funnel_device.drop_duplicates()
df_funnel_device.to_csv('./data/df_funnel_device.csv', sep='|', index=False)
df_funnel_device.head(2)

,session_id,event_type,device_type
0,1698162134395,session_end,NaN
1,1698250740429,session_start,NaN


Рассмотрим следующие группы Устройств:  
![](./pic/6.png)

![](./pic/7.png)

Выше мы отметили, что самое существенное падение конверсии происходит на переходе в *Home page viewed*  
Сравним этот этап для различных типов устройств:
- Base Funnel - все пользователи
- Null - устройство неопределено
- Iphone
- Samsung
- Other  

    - Мы можем видеть что самая низкая конверсия на рассматриваемом этапе наблюдается для Iphone и Samsung (остается 6.12% и 5% изначального трафика соответственно, в то время как глобальная цифра 9.26%, то есть существенно выше.)  
    - В категории *NULL*, то есть пользователи, чьи устройства не были идентифицированны, конверсия на этом этапе сопоставима с глобальным средним.
    - Также видно, что наилучшая конверсия в этот этап происходит в категории Other. Куда входят не только трафик мобильных устройств, но и другие. Посмотрим на Other более подробно, чтобы выявить лидеров по конверсии...


![](./pic/8.png)

- Видим, что конверсия среди пользователей других мобильных устройств сопоставима с конверсией пользователей Iphone и Samsung (то есть также низкая)
- В тоже время конверсия среди пользователей Windows, Mac и Linux (то есть Desktop платформ) существенно выше (38.45%).  
  

Изходя из этого можно сделать предположение о том, что для пользователей мобильных устройств на этом этапе возникает какой-то барьер (неверно отображаются элементы и т.п.)  

# Выводы  
Исходя из аналитики воронки продукта можно сделать следующие выводы:
- Наиболее острая "боль" продукта - низкая конверсия в Home Page среди пользователей **мобильных устройств**. Необходимо провести детальный инвестигейт клиентского пути на этом этапе.
- Также возниют вопросы о необходимости всех этапов клиентского пути (возможно стоит провести тесты для проверки гипотез о том, что те или иные элементы воронки являются лишними или неэффективными)